Import the library

In [1]:
import pyvinecopulib as pv
import numpy as np

In [2]:
# Specify pair-copulas
bicop = pv.Bicop(pv.BicopFamily.bb1, 90, [3, 2])
pcs = [[bicop, bicop], [bicop]]

# Specify R-vine matrix
mat = np.array([[1, 1, 1], [2, 2, 0], [3, 0, 0]])

# Set-up a vine copula
cop = pv.Vinecop(mat, pcs)
print(cop)

<pyvinecopulib.Vinecop>
** Tree: 0
3,1 <-> BB1 90°, parameters = 3
2
2,1 <-> BB1 90°, parameters = 3
2
** Tree: 1
3,2 | 1 <-> BB1 90°, parameters = 3
2



Showcase some methods

In [3]:
u = cop.simulate(n=10, seeds=[1, 2, 3])
fcts = [cop.pdf, cop.rosenblatt, cop.inverse_rosenblatt,
        cop.loglik, cop.aic, cop.bic]
[f(u) for f in fcts]

[array([ 44.67583001,  54.34974545,  29.84989695,  19.18643731,
        199.72159236, 944.18332674,  21.78049474,  43.78532554,
          9.10911159,  13.39339764]),
 array([[0.1638618 , 0.8170847 , 0.12668051],
        [0.90118677, 0.11345478, 0.2222099 ],
        [0.62193432, 0.12286857, 0.39604429],
        [0.6902412 , 0.78638305, 0.37455509],
        [0.13685699, 0.10872534, 0.2777274 ],
        [0.03952828, 0.35645124, 0.06597532],
        [0.48023317, 0.77472657, 0.72886036],
        [0.19787956, 0.90877192, 0.60171   ],
        [0.85523898, 0.76413881, 0.04961411],
        [0.17627362, 0.77807695, 0.03875948]]),
 array([[0.1638618 , 0.8728502 , 0.79397093],
        [0.90118677, 0.03148971, 0.30865931],
        [0.62193432, 0.28213894, 0.46366853],
        [0.6902412 , 0.28590505, 0.30879851],
        [0.13685699, 0.88594611, 0.84611164],
        [0.03952828, 0.97430917, 0.95100319],
        [0.48023317, 0.53494551, 0.47865733],
        [0.19787956, 0.84458738, 0.75111263],
    

Create a vine copula model

Different ways to fit a copula (when the families and structure are known)...

In [5]:
u = cop.simulate(n=1000, seeds=[1, 2, 3])

# Define first an object to control the fits:
#    - pv.FitControlsVinecop objects store the controls
#    - here, we only restrict the parametric family
#    - see help(pv.FitControlsVinecop) for more details
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.bb1])
print(controls)

# Create a new object an select family and parameters by fitting to data
cop2 = pv.Vinecop(mat, pcs)
cop2.select(data=u, controls=controls)
print(cop2)

# Otherwise, create directly from data
cop2 = pv.Vinecop(data=u, matrix=mat, controls=controls)
print(cop2)

<pyvinecopulib.FitControlsVinecop>
Family set: BB1
Parametric method: mle
Nonparametric method: quadratic
Nonparametric multiplier: 1
Weights: no
Selection criterion: bic
Preselect families: yes
mBIC prior probability: 0.9
Truncation level: none (default)
Tree criterion: tau
Threshold: 0
Select truncation level: no
Select threshold: no
Show trace: no
Number of threads: 1

<pyvinecopulib.Vinecop>
** Tree: 0
3,1 <-> BB1 90°, parameters = 2.92727
2.02245
2,1 <-> BB1 90°, parameters = 2.98212
2.04269
** Tree: 1
3,2 | 1 <-> BB1 90°, parameters = 2.72951
2.04219

<pyvinecopulib.Vinecop>
** Tree: 0
3,1 <-> BB1 90°, parameters = 2.92727
2.02245
2,1 <-> BB1 90°, parameters = 2.98212
2.04269
** Tree: 1
3,2 | 1 <-> BB1 90°, parameters = 2.72951
2.04219



When nothing is known, there are also two ways to fit a copula...

In [6]:
# Create a new object and select strucutre, family, and parameters
cop3 = pv.Vinecop(d=3)
cop3.select(data=u)
print(cop3)

# Otherwise, create directly from data
cop3 = pv.Vinecop(data=u)
print(cop3)

<pyvinecopulib.Vinecop>
** Tree: 0
2,1 <-> BB1 90°, parameters = 2.98212
2.04269
1,3 <-> BB6 90°, parameters = 1.54273
3.96417
** Tree: 1
2,3 | 1 <-> Gumbel 90°, parameters = 4.82967

<pyvinecopulib.Vinecop>
** Tree: 0
2,1 <-> BB1 90°, parameters = 2.98212
2.04269
1,3 <-> BB6 90°, parameters = 1.54273
3.96417
** Tree: 1
2,3 | 1 <-> Gumbel 90°, parameters = 4.82967



In [7]:
# create a C-vine structure with root node 1 in first tree, 2 in second, ...
cvine = pv.CVineStructure([4, 3, 2, 1]) 
# specify pair-copulas in every tree
tree1 = [pv.Bicop(pv.BicopFamily.gaussian, 0, [0.5]), 
         pv.Bicop(pv.BicopFamily.clayton, 0, [3]),
         pv.Bicop(pv.BicopFamily.student, 0, [0.4, 4])]
tree2 = [pv.Bicop(pv.BicopFamily.indep), 
         pv.Bicop(pv.BicopFamily.gaussian, 0, [0.5])]
tree3 = [pv.Bicop(pv.BicopFamily.gaussian)]

# instantiate C-vine copula model
cop = pv.Vinecop(cvine, [tree1, tree2, tree3])
print(cop)

<pyvinecopulib.Vinecop>
** Tree: 0
4,1 <-> Gaussian, parameters = 0.5
3,1 <-> Clayton, parameters = 3
2,1 <-> Student, parameters = 0.4
  4
** Tree: 1
4,2 | 1 <-> Independence
3,2 | 1 <-> Gaussian, parameters = 0.5
** Tree: 2
4,3 | 1,1 <-> Gaussian, parameters = 0

